<a href="https://colab.research.google.com/github/luhowle/ML-000/blob/main/Week09/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [11]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [65]:
def data_mulit(df_train):




# 构造衍生变量

    # 合并 continuous_annual_inc_joint 和 continuous_annual_inc
    df_train['continuous_annual_inc_combine'] = df_train['continuous_annual_inc_joint'] + df_train['continuous_annual_inc']


    # 合并 continuous_dti 和 continuous_dti_joint
    df_train['continuous_dti_combine'] = df_train['continuous_dti'] + df_train['continuous_dti_joint']


    # 构造还款比例

    df_train['pressure'] = df_train['continuous_installment'] / df_train['continuous_annual_inc'] 

    # 构造 continuous_dti 的二阶变量
    df_train['continuous_dti_square'] = df_train['continuous_dti'] * df_train['continuous_dti_joint'] *100

    # 构造 continuous_fico_range_high 和 continuous_fico_range_low的差值
    df_train['continuous_fico_range_change'] = df_train['continuous_fico_range_high'] - df_train['continuous_fico_range_low']
    df_train['continuous_last_fico_range_change'] = df_train['continuous_last_fico_range_high'] - df_train['continuous_last_fico_range_low']

    # 构造 continuous_int_rate 的二阶变量
    df_train['continuous_int_rate_square'] = df_train['continuous_int_rate']*df_train['continuous_int_rate']

    # 构造  贷款*利率 / 申报收入
    df_train['continuous_dti2'] = df_train['continuous_funded_amnt']*df_train['continuous_int_rate'] * 0.01 / df_train['continuous_annual_inc_combine']


    # 构造 continuous_funded_amnt 和 continuous_funded_amnt_inv的差值
    df_train['continuous_funded_real'] = df_train['continuous_funded_amnt'] - df_train['continuous_funded_amnt_inv']
 
    return df_train

In [66]:
df_train = pd.read_csv("/train_final.csv")
df_test = pd.read_csv("/test_final.csv")

In [71]:

target = np.array(df_train.loan_status)


# 训练过程

# 无衍生变量
data1 = np.array(df_train.drop(labels=['loan_status'],axis=1))
X_train, X_test, y_train, y_test = train_test_split(data1, target, test_size=0.2)
gbm = lgb.LGBMClassifier(objective='binary', num_leaves=31, learning_rate=0.05, n_estimators=100)
gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='l2', early_stopping_rounds=100)
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
print('Training The accuracy of prediction is:', accuracy_score(y_test, y_pred) )


target_test = np.array(df_test.loan_status)
data_test = np.array(df_test.drop(labels=['loan_status'],axis=1))
y_pred = gbm.predict(data_test, num_iteration=gbm.best_iteration_)

print('1st Training The accuracy of prediction is:', accuracy_score(target_test, y_pred) )

result1 = accuracy_score(target_test, y_pred)


# 有衍生变量
# 构造变量
df_train_multi = data_mulit(df_train)

data2 = np.array(df_train_multi.drop(labels=['loan_status'],axis=1))
X_train, X_test, y_train, y_test = train_test_split(data2, target, test_size=0.2)
gbm2 = lgb.LGBMClassifier(objective='binary', num_leaves=31, learning_rate=0.05, n_estimators=100)
gbm2.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='l2', early_stopping_rounds=100)
y_pred = gbm2.predict(X_test, num_iteration=gbm.best_iteration_)
print('2nd Training The accuracy of prediction is:', accuracy_score(y_test, y_pred) )

df_test_multi = data_mulit(df_test)

data_test = np.array(df_test_multi.drop(labels=['loan_status'],axis=1))
y_pred = gbm2.predict(data_test, num_iteration=gbm.best_iteration_)
print('Training The accuracy of prediction is:', accuracy_score(target_test, y_pred) )

result2 = accuracy_score(target_test, y_pred)
print(result1)
print(result2)
improve = (result2- result1)

print("improvement = ",improve)


[1]	valid_0's l2: 0.153538	valid_0's binary_logloss: 0.478534
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's l2: 0.144173	valid_0's binary_logloss: 0.452446
[3]	valid_0's l2: 0.135804	valid_0's binary_logloss: 0.430049
[4]	valid_0's l2: 0.128276	valid_0's binary_logloss: 0.410414
[5]	valid_0's l2: 0.121536	valid_0's binary_logloss: 0.393112
[6]	valid_0's l2: 0.115468	valid_0's binary_logloss: 0.377649
[7]	valid_0's l2: 0.109981	valid_0's binary_logloss: 0.363707
[8]	valid_0's l2: 0.105033	valid_0's binary_logloss: 0.351086
[9]	valid_0's l2: 0.100576	valid_0's binary_logloss: 0.339636
[10]	valid_0's l2: 0.0965886	valid_0's binary_logloss: 0.329268
[11]	valid_0's l2: 0.0929688	valid_0's binary_logloss: 0.319733
[12]	valid_0's l2: 0.0896938	valid_0's binary_logloss: 0.310978
[13]	valid_0's l2: 0.0867306	valid_0's binary_logloss: 0.302918
[14]	valid_0's l2: 0.0840532	valid_0's binary_logloss: 0.295495
[15]	valid_0's l2: 0.0816295	valid_0's binary_logloss: 0.28